In [3]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from keras import layers
from keras import losses

In [44]:
import pandas as pd

train_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], 
                    # dtype={'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party_id': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

test_df = pd.read_csv('/Users/jacobjohnson/data_sets/congressional_tweet_test_data.csv', names=['id', 'favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party'], 
                    # dtype={'id': int, 'favorite_count': int, 'full_text': str, 'hashtags': str, 'retweet_count': int, 'year': int, 'party': str}, 
                    skipinitialspace=True, skiprows=1, sep=',')

train_df.pop('year')
test_df.pop('year')

# test_df.head()
train_df.head()

# target variable
# party = train_df.pop('party_id')



,favorite_count,full_text,hashtags,retweet_count,year,party_id
0,0,"b""RT @KUSINews: One of our longtime viewers wa...",KUSI,10,2017.0,R
1,258,"b""Today I'm urging the @CDCgov to immediately ...",Coronavirus,111,2020.0,R
2,0,"b'Tomorrow, #MO03 seniors graduate from Calvar...",MO03,2,2014.0,R
3,9,b'Congrats to #TeamUSA and Canton Native @JGre...,TeamUSA WorldJuniors,3,2017.0,R
4,3,b'Pleased to support @amergateways at their Ju...,ImmigrantHeritageMonth,3,2019.0,D


In [48]:
train, val, test = np.split(train_df.sample(frac=1), [int(0.8*len(train_df)), int(0.9*len(train_df))])

print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

474242 training examples
59280 validation examples
59281 test examples


In [46]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  names = df.pop('party_id')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), names))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [49]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

/var/folders/nf/brcqg8zx7xg31hjn_xbd2jc80000gp/T/ipykernel_18992/2315459290.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


In [51]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of hashtags:', train_features['hashtags'])
print('A batch of targets:', label_batch )

Every feature: ['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id']
A batch of hashtags: tf.Tensor(
[[b'CA10']
 [b'PerkinsLoan middleclass']
 [b'dontdoublemyrate NN13']
 [b'DREAMAct']
 [b'MaxineWaters']], shape=(5, 1), dtype=string)
A batch of targets: tf.Tensor([b'R' b'D' b'D' b'D' b'D'], shape=(5,), dtype=string)
